In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
def load_latest_indicator(path, indicator_name, value_col_name):
    """
    Wczytuje najnowsze dostępne dane dla jednego wskaźnika z pliku CSV.
    Pomija wiersze z break-in-time-series (OBS_FLAG='b') i brakujące wartości.
    
    Args:
        path (str): ścieżka do pliku CSV
        indicator_name (str): nazwa wskaźnika w kolumnie 'Urban audit indicator'
        value_col_name (str): nowa nazwa kolumny z wartościami
    
    Returns:
        pd.DataFrame: DataFrame z kolumnami ['Geopolitical entity (declaring)', value_col_name]
    """
    df = pd.read_csv(path, delimiter=',')
    df = df[df['Urban audit indicator'] == indicator_name].copy()

    # Pomijamy break-in-time-series i brakujące wartości
    if 'OBS_FLAG' in df.columns:
        df = df[df['OBS_FLAG'] != 'b']
    df = df.dropna(subset=['OBS_VALUE'])

    # Sortujemy malejąco po roku i wybieramy najnowszy wiersz dla każdego miasta
    df = df.sort_values(['Geopolitical entity (declaring)', 'TIME_PERIOD'], ascending=[True, False])
    df = df.drop_duplicates(subset=['Geopolitical entity (declaring)'], keep='first')

    # Zmieniamy nazwę kolumny z wartościami
    return df[['Geopolitical entity (declaring)', 'OBS_VALUE']].rename(columns={'OBS_VALUE': value_col_name})

In [3]:
def add_feature(df, feature_df):
    """
    Dołącza dodatkowe dane (feature) do głównego DataFrame po nazwach miast.
    
    Args:
        df (pd.DataFrame): główny DataFrame
        feature_df (pd.DataFrame): DataFrame z dodatkowymi kolumnami do połączenia
    
    Returns:
        pd.DataFrame: zaktualizowany główny DataFrame
    """
    return df.merge(
        feature_df,
        on='Geopolitical entity (declaring)',
        how='left'
    )

## Wczytanie nazw miast i całkowitej populacji

In [4]:
# Wczytanie wszystkich danych o populacji
pop_total = pd.read_csv('../data/raw/01_population.csv', delimiter=',')

# Wybór wierszy z całkowitą populacją i pominięcie break-in-time-series
pop_total = pop_total[
    (pop_total['Urban audit indicator'] == 'Population on the 1st of January, total') &
    (pop_total['OBS_FLAG'] != 'b')
]

# Sortowanie malejąco po roku i wybór najnowszych danych dla każdego miasta
pop_total = pop_total.sort_values(['Geopolitical entity (declaring)', 'TIME_PERIOD'], ascending=[True, False])
pop_latest = pop_total.drop_duplicates(subset=['Geopolitical entity (declaring)'], keep='first')

# Tworzymy główny DataFrame
df = pop_latest[['Geopolitical entity (declaring)', 'OBS_VALUE']].rename(columns={'OBS_VALUE': 'Population'})

## Dodanie informacji o strukturze wiekowej

In [5]:
age_dep = pd.read_csv('../data/raw/02_population_structure.csv', delimiter=',')

In [6]:
# Lista interesujących wskaźników
indicators = [
    'Age dependency ratio (population aged 0-19 and 65 and more to population aged 20-64)',
    'Young-age dependency ratio (population aged 0-19 to population 20-64 years)',
    'Old age dependency ratio (population 65 and over to population 20 to 64 years)'
]

# Wybieramy tylko te wskaźniki oraz medianę wieku
age_dep = age_dep[age_dep['Urban audit indicator'].isin(indicators + ['Median population age'])]

# Pomijamy break-in-time-series i brakujące wartości
if 'OBS_FLAG' in age_dep.columns:
    age_dep = age_dep[age_dep['OBS_FLAG'] != 'b']
age_dep = age_dep.dropna(subset=['OBS_VALUE'])

# Sortowanie malejąco po roku dla każdego miasta/wskaźnika
age_dep = age_dep.sort_values(['Geopolitical entity (declaring)', 'Urban audit indicator', 'TIME_PERIOD'],
                              ascending=[True, True, False])

# Wybór najnowszych wartości dla każdego miasta/wskaźnika
age_dep = age_dep.drop_duplicates(subset=['Geopolitical entity (declaring)', 'Urban audit indicator'], keep='first')

# Pivotowanie: jeden wiersz = jedno miasto, kolumny = wskaźniki
age_dep_wide = age_dep.pivot(index='Geopolitical entity (declaring)',
                             columns='Urban audit indicator', values='OBS_VALUE').reset_index()

# Zmiana nazw kolumn na bardziej czytelne
age_dep_wide = age_dep_wide.rename(columns={
    indicators[0]: 'Age_dependency_ratio',
    indicators[1]: 'Young_dependency_ratio',
    indicators[2]: 'Old_dependency_ratio',
    'Median population age': 'Median_age'
})

# Dodanie do głównego DataFrame
df = add_feature(df, age_dep_wide)

## Wczytanie informacji o odsetku obcokrajowców

In [7]:
foreigners = load_latest_indicator(
    path='../data/raw/03_foreigners.csv',
    indicator_name='Foreigners as a proportion of population',
    value_col_name='Share_foreigners'
)

df = add_feature(df, foreigners)

In [8]:
df

,Geopolitical entity (declaring),Population,Age_dependency_ratio,Median_age,Old_dependency_ratio,Young_dependency_ratio,Share_foreigners
0,'s-Gravenhage (greater city),813669,61.1,37.0,26.0,35.2,19.4
1,'s-Hertogenbosch,160757,63.8,40.0,29.8,34.0,6.5
2,A Coruña (greater city),247350,67.9,48.3,40.9,27.0,7.6
3,Aachen,252136,54.1,37.0,28.1,25.5,22.3
4,Aberdeen,228180,53.0,37.1,23.7,29.2,16.5
...,...,...,...,...,...,...,...
827,České Budějovice,93426,68.4,43.4,35.5,32.9,3.0
828,Łomża,60468,55.5,40.8,24.7,30.9,0.2
829,Łódź,664860,65.2,44.9,38.1,27.1,0.2
830,Šiauliai,110331,62.8,43.0,31.8,31.0,11.5
